# 과제물 자동 평가 시스템

In [1]:
import os
import sys
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 모듈 경로 추가
sys.path.append(os.path.abspath('../'))

# 모듈형 위젯 라이브러리 import
from widgets_core import (
    ChecklistCreationManager,
    AssignmentEvaluationManager,
    LlmCallManager
)

## 1. 체크리스트 생성 위젯

체크리스트 생성 기능을 위한 위젯 인터페이스를 생성합니다.

In [2]:
# 체크리스트 생성 매니저 생성
checklist_manager = ChecklistCreationManager()

# 위젯 표시
checklist_manager.display_all()

#### 예시
학년 : 1학년 \
과목 이름 : 국어 \
수행평가 제목 : 비혼주의자에 대한 본인의 의견\
수행평가 유형 : 찬성반대\
수행평가 설명 : 여러분은 결혼을 꼭 해야 한다고 생각하시나요? 아니면 결혼은 선택의 문제라고 생각하시나요? 비혼주의에 대해 찬성하는 입장 반대를 선택하여 자신의 의견을 작성해주세요. 그리고 그렇게 생각한 이유에 대해서 작성해주세요.

## 2. 과제 평가 위젯

학생 과제 평가 기능을 위한 위젯 인터페이스를 생성합니다.

#### 예시
학년 : 1학년 \
과목 이름 : 국어 \
수행평가 제목 : 비혼주의자에 대한 본인의 의견\
수행평가 유형 : 찬성반대\
수행평가 설명 : 여러분은 결혼을 꼭 해야 한다고 생각하시나요? 아니면 결혼은 선택의 문제라고 생각하시나요? 비혼주의에 대해 찬성하는 입장 반대를 선택하여 자신의 의견을 작성해주세요. 그리고 그렇게 생각한 이유에 대해서 작성해주세요.

In [3]:
# 과제 평가 매니저 생성
evaluation_manager = AssignmentEvaluationManager()

# 위젯 표시
evaluation_manager.display_all()

## 3. 단순 LLM API 콜 위젯
이 위젯은 단순한 LLM API 호출을 위한 것입니다.

In [5]:
# Create and display the widget manager
manager = LlmCallManager()
manager.display_all()